# Interactive Visualization: iPyWidgets


Lesson Goals

    Learn about iPyWidgets.
    Introduce the interact decorator.
    Create visualizations with interactive sliders.
    Create visualizations with interactive drop-down boxes.
    Create visualizations with interactive check boxes.
    Create visualizations with interactive text boxes.

Introduction

In the last lesson, we covered how to make interactive, Javascript-based visualizations using Python and the plotly and cufflinks libraries. This provided us with the ability to hover over the plot points in the chart to see the values, and it also enabled us to include and exclude groups from the visualization by simply clicking on them in the legend.

The interactivity doesn't have to stop there. In this lesson, we are going learn how to make our visualizations even more interactive via the use of widgets such as sliders, drop-down boxes, check boxes, and text boxes. These widgets are going to control aspects of the information that get passed to our visualizations. Using these widgets to change values is going to cause the visualizations to change themselves. We will be using these widgets to extend the interactivity of charts generated with plotly and cufflinks.

To incorporate widgets into our visualizations, we will need to ensure that we have the iPyWidgets library installed.

$ pip install ipywidgets

Let's also go ahead and import everything we are going to need for this lesson.

In [1]:
import pandas as pd
import lineup_widget
from IPython.display import display
import plotly.plotly as py
import cufflinks as cf
from ipywidgets import interact

cf.go_offline()

Finally, for this lesson we will continue using the same churn data set that we used for the plotly and cufflinks lesson, so make sure that is imported as well.

In [2]:
df = pd.read_csv('../data/churn.csv')
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,MonthLevel,TotalLevel,TenureLevel,ChurnBinary
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,Month-to-month,Yes,Electronic check,29.85,29.85,No,Low,Very Low,New,0.0
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,One year,No,Mailed check,56.95,1889.50,No,Low,Moderate,Loyal,0.0
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,Month-to-month,Yes,Mailed check,53.85,108.15,Yes,Low,Very Low,New,1.0
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,One year,No,Bank transfer (automatic),42.30,1840.75,No,Low,Moderate,Loyal,0.0
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,Month-to-month,Yes,Electronic check,70.70,151.65,Yes,Moderate,Very Low,New,1.0


# The Interact Decorator

IPyWidgets library has a variety of functionality for creating interactive widgets, but the interact decorator is both the easiest way to get started and the most useful, so we will be focusing it for this lesson.

The interact decorator accepts a few different types of inputs, and the format of those inputs determines the type of widget that is displayed.

    Slider: a numeric value, (min, max), or (min, max, step)
    Drop-Down Box: a list or dictionary
    Check Box: True or False values
    Text Box: a string enclosed in quotes

We will see examples of each of these in the sections below, both individually and then all together.


# Interactive Sliders

One of the most useful types of widgets you can use for numeric variables is the slider. Sliders allow you to modify numeric inputs that are fed to your visualizations.

We can see a basic example of this with the histogram below. One of the challenges of working with histograms is determining an appropriate number of bins. Widgets can help us with this by allowing us to quickly and easily view different numbers of bins without having to manually input them into our code and regenerate the visualization.

We can use the interact decorator to create a slider widget where we specify a range of bins (from as few as 8 to as many as there are unique tenures). Then, we can simply define a hist function (it really doesn't matter what you name it) that accepts the dynamic number of bins and plugs them in to a plotly histogram so that the number of bins in the visualization updates as you modify the values with the slider.

In [3]:
@interact(bins = (8, len(df['tenure'].unique())))
def hist(bins):
    df['tenure'].iplot(kind='hist', bins=bins, title='Tenure Distribution')

If we slide the slider all the way to the right, the number of bins increases to 72, which is the number of unique tenure values there are in our data set.


# Interactive Drop-Down Boxes

Another useful widget when you have the same visualization that you'd like to view for either different fields or for different field values is the drop-down box. To generate one, we can pass a list to our interact decorator consisting of the fields we would like to be able to choose from. We can then define a linechart function that accepts the input we have chosen from the drop-down box, pivots our data set so that the columns represent the categorical values in the field we have chosen, and then generates a plotly line chart with lines showing changes in churn rate for each category.



In [4]:
@interact(Selection=['gender', 'SeniorCitizen', 'Partner', 
                     'Dependents', 'InternetService', 'PaymentMethod'])
def linechart(Selection):
    data = df.pivot_table(values='ChurnBinary', columns=Selection,
                            index='tenure', aggfunc='mean').reset_index()

    data.iplot(kind='line', x='tenure', xTitle='Tenure', 
               yTitle='Avg. Churn Rate', title='Avg. Churn Rate by ' + Selection.title())

As you can see, having the check box where we can choose the field we want to see saves us from having to jump back into the code, create a pivot table for the specific field, and then regenerate the visualization. This has the potential to save us a significant amount of time in our data exploration workflow.

Let's look at another example where we use multiple checkboxes to filter our data down to a point where we can investigate it at a granular level.

Below, we are creating 4 different drop-down boxes containing the unique categorical values in the gender, Partner, InternetService, and PaymentMethod fields respectively. We then define a scatter function where we filter our data down by the values chosen in each of the drop-down boxes and then generate a scatter plot that plots how much each customer is charged by their tenure and color codes the points by the type of contract they have.

In [5]:
@interact(Gender=list(df['gender'].unique()), 
          Partner=list(df['Partner'].unique()),
          Internet=list(df['InternetService'].unique()), 
          Payment=list(df['PaymentMethod'].unique())
         )

def scatter(Gender, Partner, Internet, Payment):
    data = df[(df['gender']==Gender) & 
              (df['Partner']==Partner) & 
              (df['InternetService']==Internet) & 
              (df['PaymentMethod']==Payment)]

    data.iplot(kind='scatter', x='tenure', y='MonthlyCharges', 
               categories='Contract', text='customerID', 
               xTitle='Tenure', yTitle='Monthly Charges',
               title='Charges vs. Tenure')

Note that we added a text argument to our iplot method that will show us each customer's customerID in addition to the monthly amount they are paying when we hover over the data points.



# Interactive Check Boxes

Interactive check boxes can also help you explore a data set, especially when there are binary fields whose impact you'd like to visualize. The way to do this is to map whether the check box is checked to a corresponding condition for the binary field. Once that is done, you'd just need to filter the data set based on those conditions.

Below is an example that does exactly this. The interact decorator has two True/False arguments which it will render as check boxes - one for Senior and one for PhoneService. Inside our barchart function, we write some conditional statements that will translate those True/False options into conditions that we can use to filter our data. We then apply those filters and group the data by PaymentMethod, calculating the average churn rate for each. Finally, we generate a bar chart that displays the average churn rate for each payment method based on the filters applied via the check boxes.

In [6]:
@interact(Senior=False, PhoneService=True)

def barchart(Senior, PhoneService):
    if Senior==True:
        senior = df['SeniorCitizen']==1
    else:
        senior = df['SeniorCitizen']==0
    
    if PhoneService==True:
        phone = df['PhoneService']=='Yes'
    else:
        phone = df['PhoneService']=='No'
    
    data = df[(senior) & (phone)].groupby('PaymentMethod').agg({'ChurnBinary':'mean'}).reset_index()
    
    data.iplot(kind='bar', x='PaymentMethod', xTitle='Payment Method',
               yTitle='Avg. Churn Rate', color='blue', 
               title='Churn Rate by Payment Method')

The code above generates the following bar chart visualization. By default, the PhoneService check box is checked because we initially set it to True.

If we also check the Senior checkbox, the visualization instantly updates to look like the bar chart below.


# Interactive Text Boxes

The last type of widget we are going to cover in this lesson is the interactive text box. These are good for filtering when you have categorical variables that are different but have some string in common. For example, the PaymentMethod field in our data set has four unique values.



In [7]:
df['PaymentMethod'].unique()


array(['Electronic check', 'Mailed check', 'Bank transfer (automatic)',
       'Credit card (automatic)'], dtype=object)

Notice that there are similarities between pairs of these unique categories - two of them have the string 'check' in common and the other two have the string 'automatic' in common. We can use a text box to provide a flexible means by which we can visualize any of these options based on the unique strings they contain or groups of them based on their co-occurring strings.

To do this, we will once again use the interact decorator, passing a Payment argument with a blank string. This will create a text box into which we can type any string we want. In our chart function, we will then filter our data set to just the payment methods that contain the string typed into the text box. We then aggregate, sort, and visualize as a bar chart.

In [8]:
@interact(Payment='')

def chart(Payment):
    data = df[df['PaymentMethod'].str.contains(Payment)]
    data = data.groupby('TenureLevel').agg({'MonthlyCharges':'sum'}).reset_index()
    custom_dict = {'New': 0, 'Regular': 1, 'Loyal': 2, 'Very Loyal' : 3}  
    data = data.iloc[data['TenureLevel'].map(custom_dict).argsort()].set_index('TenureLevel')
    
    data.iplot(kind='bar', xTitle='Values')

When we type 'check' into the text box, the chart updates and shows us the total Monthly Charges the company collects from each tenure level. These are all relatively even.

However, when we type 'auto' into the text box, the chart shows that more loyal customers tend to use these payment methods to pay their bills.